#**Upload File**

In [0]:
from google.colab import files
uploaded = files.upload()

Saving merged_final.xlsx to merged_final.xlsx


#**Import Module**

In [0]:
import openpyxl

In [0]:
import pandas as pd

In [0]:
import gc

#**Open Workbook**

In [0]:
wb = openpyxl.load_workbook('merged_final.xlsx')

#**Read and Process Data**

In [0]:
def pair_to_vector(idx, length):
    res = [0] * length
    for i in idx:
        res[i] = 1
    return res

In [0]:
ws = wb.active
MAXROW = ws.max_row
MAXCOL = ws.max_column

keywords = []
papers = []
keyword_paper_pair = []
curr = 0
curr_idx = -1

for r in ws.rows:
    # ignore header
    if(r[0].row == 1):
        continue
    # halt loop
    if(r[0].row > MAXROW):
        break
    # update cursor
    if(isinstance(r[2].value, str)):
        curr = r[2].value
        papers.append(curr)
        curr_idx += 1
    # row corresponds to a keyword
    if(isinstance(r[3].value, str)):
        # duplicate exists
        if(r[3].value.lower() in keywords):
            keyword_paper_pair[keywords.index(r[3].value.lower())].append(curr_idx)
        # duplicate doesn't exist
        else:
            keywords.append(r[3].value.lower())
            keyword_paper_pair.append([curr_idx])

# encode keyword-paper pair to vector
keyword_vector = []
for i in range(len(keyword_paper_pair)):
    keyword_vector.append(pair_to_vector(keyword_paper_pair[i], curr_idx + 1))

wb.close()

#**Write Data to CSV**

In [0]:
gc.collect()

0

In [0]:
df = pd.DataFrame(keyword_vector)
df = df.transpose()
df.index = papers
df.columns = keywords

In [0]:
len(papers)

5664

In [0]:
len(keyword_vector)

9942

In [0]:
df.to_csv("final_paper-keyword_mat.csv", mode='w')

#**Sort by Frequency**

In [0]:
freq = df.sum(axis=0).tolist()
freq_mat = []
for i in range(len(freq)):
    freq_mat.append([keywords[i], freq[i]])
freq_mat.sort(key=lambda x: x[1])
freq_mat.reverse()
pd.DataFrame(freq_mat).to_csv("final_paper-keyword_freq.csv", mode="w", header=False, index=False)

#**Download File**

In [0]:
files.download("final_paper-keyword_freq.csv")

In [0]:
files.download("final_paper-keyword_mat.csv")